# Product Optimization

## Set-up:

In [440]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools

## Question 1:

To obtain our candidate's share, price, margin, and EPPP, we first need to derive four critical inputs: <br>
- 1) Description of candidate product and incumbent products
- 2) Preference parameters
- 3) Cost structure and dollar values for each price level
- 4) "c" value for logit probability formula

Let's first derive critical input 1 (*candidate and incumbent products*) below.

In [441]:
df_1 = pd.DataFrame(np.array([[1, 2, 1], [3, 2, 2], [2, 2, 2],[3, 2, 3],[3, 2, 3],[1, 2, 3]]), columns=['Prod1', 'Prod2', 'Prod3'], index=['Pr', 'Ti', 'Cp', 'Cl', 'Co', 'Br'])

df_1

,Prod1,Prod2,Prod3
Pr,1,2,1
Ti,3,2,2
Cp,2,2,2
Cl,3,2,3
Co,3,2,3
Br,1,2,3


Let's derive critical input 2 (*preference parameters*) next.

In [442]:
xl_file = pd.ExcelFile('mugs-preference-parameters-full.xlsx')
df_2 = xl_file.parse("mugs-full")
df_2

,Cust,pPr30,pPr10,pPr05,pIn0.5,pIn1,pIn3,pCp12,pCp20,pCp32,...,pCnLk,pBrA,pBrB,pBrC,IPr,Iin,ICp,ICl,Icn,IBr
0,1,1,3,7,1,3,7,1,7,6,...,7,7,5,1,5,8,32,7,34,14
1,2,1,6,7,1,4,7,1,7,6,...,7,7,5,1,6,3,17,10,51,15
2,3,1,6,7,1,4,7,7,4,1,...,7,1,3,7,55,5,5,8,16,9
3,4,1,6,7,1,4,7,7,1,3,...,7,7,1,5,12,9,3,40,20,16
4,5,1,4,7,1,4,7,1,7,2,...,7,3,1,7,56,6,19,9,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,1,3,7,1,6,7,1,7,6,...,7,4,1,7,54,13,6,10,7,10
307,308,1,3,7,1,6,7,1,7,6,...,7,7,1,4,8,24,4,39,15,11
308,309,1,6,7,1,5,7,1,7,3,...,7,7,3,1,12,34,16,14,13,11
309,310,1,3,7,1,5,7,1,7,1,...,7,1,5,7,57,5,5,17,5,10


Then we derive critical input 3 (*cost structure and price levels*) next.

In [443]:
df_prices = pd.DataFrame(np.array([[30], [10], [5]]), columns=['Prices'], index=["P1", "P2", "P3"])
df_costs = pd.DataFrame(np.array([[0.5], [1], [3], [1], [2.6], [2.8], [1], [2.2], [3.0], [0.5], [0.8], [1]]), columns=['Costs'], index=["T1", "T2", "T3", "Ca1", "Ca2", "Ca3", "Cl1", 'Cl2', "Cl3", "Co1", "Co2", "Co3"])

print(df_prices)
print(df_costs)

    Prices
P1      30
P2      10
P3       5
     Costs
T1     0.5
T2     1.0
T3     3.0
Ca1    1.0
Ca2    2.6
Ca3    2.8
Cl1    1.0
Cl2    2.2
Cl3    3.0
Co1    0.5
Co2    0.8
Co3    1.0


Finally, we derive our critical input 4 (*c value*) which we have already computed as 0.0139.

In [444]:
c_value = 0.0139

Now that we have our four critical inputs, let's first compute the product of importance and preference level for each attribute level.

In [445]:
df_IPR = pd.DataFrame()

# Price attribute
for i in range(1, 4):
    df_IPR['P' + str(i)] = df_2.iloc[:, i] * df_2[' IPr']
    
# Time attribute
for i in range(1, 4):
    df_IPR['T' + str(i)] = df_2.iloc[:, i+3] * df_2['Iin']
    
# Capacity attribute
for i in range(1, 4):
    df_IPR['Ca' + str(i)] = df_2.iloc[:, i+6] * df_2[' ICp']
    
# Cleanability attribute
for i in range(1, 4):
    df_IPR['Cl' + str(i)] = df_2.iloc[:, i+9] * df_2[' ICl']
    
# Containment attribute
for i in range(1, 4):
    df_IPR['Co' + str(i)] = df_2.iloc[:, i+12] * df_2['Icn']
    
# Brand attribute
for i in range(1, 4):
    df_IPR['Br' + str(i)] = df_2.iloc[:, i+15] * df_2[' IBr']

df_IPR

,P1,P2,P3,T1,T2,T3,Ca1,Ca2,Ca3,Cl1,Cl2,Cl3,Co1,Co2,Co3,Br1,Br2,Br3
0,5,15,35,8,24,56,32,224,192,7,14,49,34,170,238,98,70,14
1,6,36,42,3,12,21,17,119,102,10,60,70,51,357,357,105,75,15
2,55,330,385,5,20,35,35,20,5,8,24,56,16,80,112,9,27,63
3,12,72,84,9,36,63,21,3,9,40,200,280,20,100,140,112,16,80
4,56,224,392,6,24,42,19,133,38,9,27,63,0,0,0,33,11,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,54,162,378,13,78,91,6,42,36,10,70,70,7,35,49,40,10,70
307,8,24,56,24,144,168,4,28,24,39,234,273,15,30,105,77,11,44
308,12,72,84,34,170,238,16,112,48,14,28,98,13,39,91,77,33,11
309,57,171,399,5,25,35,5,35,5,17,68,119,5,20,35,10,50,70


Now, let's compute the utility for each product. First, we need to create a dummy variable matrix based on the attribute levels from our critical input 1 (candidate and incumbent products).

In [446]:
# Initialize dummy variable matrix dataframe
dummy = np.zeros(shape=(18,3))
df_dummy = pd.DataFrame(dummy, columns = ['Prod1','Prod2','Prod3'])

# Create dummy variable vectors for each product separately
prod_1 = [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
prod_2 = [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
prod_3 = [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]

df_dummy['Prod1'] = prod_1
df_dummy['Prod2'] = prod_2
df_dummy['Prod3'] = prod_3

df_dummy

,Prod1,Prod2,Prod3
0,1,0,1
1,0,1,0
2,0,0,0
3,0,0,0
4,0,1,1
5,1,0,0
6,0,0,0
7,1,1,1
8,0,0,0
9,0,0,0


Now, let's find the utilities of each product by doing matrix multiplication of the dummy variable matrix and the attribute level matrix. After finding utilities, we multiply by our "c" value and then exponentiate. We can then find the sum of the exponentials which we use to divide each exponential and get the purchase probability.

In [447]:
# Calculate utilities multiplied by our "c" value and then exponentiated
utility = np.dot(df_IPR,df_dummy)
c_utility = np.multiply(utility,c_value)
exp_c_u = np.exp(c_utility)

# Use a for loop to find the sum of exponentiated utility values
sum_exp = np.zeros(shape=(311,1)) 

for i in range(len(sum_exp)):
    for j in range(3):
        sum_exp[i] += exp_c_u[i][j]

# Use a for loop to get purchase probabilities and then compute average probabilities
purch_prob = np.zeros(shape=(311,3))

for i in range(len(sum_exp)):
    for j in range(3):
        purch_prob[i][j] = exp_c_u[i][j] / sum_exp[i]

avg_prob = np.mean(purch_prob, axis=0)
cand_share = avg_prob[2]
print("Candidate share:", cand_share)

Candidate share: 0.19452610767391657


As we can see above, our candidate product has a share of about 0.195.

We can compute the price and cost of our candidate product next in order to find both our margin and EPPP (expected profit per person) for the proposed market scenario.

In [448]:
# Compute price
p_array = df_prices.to_numpy()
p_array = np.reshape(p_array, 3)

p_dummy_arr = df_dummy.iloc[0:3, 2:3].to_numpy()
p_dummy_arr = np.reshape(p_dummy_arr, 3)

cand_price = np.dot(p_array,p_dummy_arr)

# Compute cost
c_array = df_costs.to_numpy()
c_array = np.reshape(c_array, 12)

c_dummy_arr = df_dummy.iloc[3:15, 2:3].to_numpy()
c_dummy_arr = np.reshape(c_dummy_arr, 12)

cand_cost = np.dot(c_array,c_dummy_arr)

# Compute margin
cand_margin = cand_price - cand_cost

# Compute EPPP
cand_eppp = round(cand_margin * cand_share, 3)

# Report our results below
print("Candidate price:", cand_price)
print("Candidate cost:", cand_cost)
print("Candidate margin:", cand_margin)
print("Candidate EPPP:", cand_eppp)

Candidate price: 30
Candidate cost: 7.6
Candidate margin: 22.4
Candidate EPPP: 4.357


## Question 2:

First, let's create a table enumerating all possibilities of product combinations in lexical order.

In [449]:
combinations = [['$30', '$10', '$5'], 
     ['0.5 hrs', '1 hrs', '3 hrs'], 
     ['12 oz', '20 oz', '32 oz'],
     ['Difficult', 'Fair', 'Easy'],
     ['Slosh resistant', 'Spill resistant', 'Leak resistant']]

combo_list = list(itertools.product(*combinations))
df_combo = pd.DataFrame(combo_list, columns = ['Price', 'Time', 'Capacity', 'Cleanability', 'Containment'])
df_combo['Share'], df_combo['price'], df_combo['Margin'], df_combo['EPPP'] = [np.nan, np.nan, np.nan, np.nan]
df_combo.index += 1 

df_combo

,Price,Time,Capacity,Cleanability,Containment,Share,price,Margin,EPPP
1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant,NaN,NaN,NaN,NaN
2,$30,0.5 hrs,12 oz,Difficult,Spill resistant,NaN,NaN,NaN,NaN
3,$30,0.5 hrs,12 oz,Difficult,Leak resistant,NaN,NaN,NaN,NaN
4,$30,0.5 hrs,12 oz,Fair,Slosh resistant,NaN,NaN,NaN,NaN
5,$30,0.5 hrs,12 oz,Fair,Spill resistant,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
239,$5,3 hrs,32 oz,Fair,Spill resistant,NaN,NaN,NaN,NaN
240,$5,3 hrs,32 oz,Fair,Leak resistant,NaN,NaN,NaN,NaN
241,$5,3 hrs,32 oz,Easy,Slosh resistant,NaN,NaN,NaN,NaN
242,$5,3 hrs,32 oz,Easy,Spill resistant,NaN,NaN,NaN,NaN


Next, let's create a critical input 1 (candidate products) for all our possible products for Brand C.

In [450]:
product_combos = [[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]
product_combo_list = list(itertools.product(*product_combos))
df_product_combo = pd.DataFrame(product_combo_list, columns = ['Price', 'Time', 'Capacity', 'Cleanability', 'Containment'])
df_product_combo['Brand'] = 3

df_product_combo

,Price,Time,Capacity,Cleanability,Containment,Brand
0,1,1,1,1,1,3
1,1,1,1,1,2,3
2,1,1,1,1,3,3
3,1,1,1,2,1,3
4,1,1,1,2,2,3
...,...,...,...,...,...,...
238,3,3,3,2,2,3
239,3,3,3,2,3,3
240,3,3,3,3,1,3
241,3,3,3,3,2,3


Now, we can create a dummy variable matrix for all of our possible product combinations.

In [451]:
# Create attribute level array
attribute_levels = np.array([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3])
attribute_levels

# Create dummy variable matrix for all possible products
dummy_all = np.zeros(shape=(18))
df_dummy_all = pd.DataFrame(dummy_all)

for i in range(243):
    df_dummy_all['Cand_' + str(i+1)] = 0
    
df_dummy_all = df_dummy_all.drop(df_dummy_all.iloc[:,0], axis=1)

# Iterate through price level vector and critical input 1 for all candidate products
for i in range(243):
    for j in range(3):
        if attribute_levels[j] == df_product_combo.iloc[i,0]:
            df_dummy_all.iloc[j,i] = 1
        else:
            df_dummy_all.iloc[j,i] = 0

# Iterate through time level vector and critical input 1 for all candidate products
for i in range(243):
    for j in range(3,6):
        if attribute_levels[j] == df_product_combo.iloc[i,1]:
            df_dummy_all.iloc[j,i] = 1
        else:
            df_dummy_all.iloc[j,i] = 0
            
# Iterate through capacity level vector and critical input 1 for all candidate products
for i in range(243):
    for j in range(6,9):
        if attribute_levels[j] == df_product_combo.iloc[i,2]:
            df_dummy_all.iloc[j,i] = 1
        else:
            df_dummy_all.iloc[j,i] = 0
            
# Iterate through cleanability level vector and critical input 1 for all candidate products
for i in range(243):
    for j in range(9,12):
        if attribute_levels[j] == df_product_combo.iloc[i,3]:
            df_dummy_all.iloc[j,i] = 1
        else:
            df_dummy_all.iloc[j,i] = 0

# Iterate through containment level vector and critical input 1 for all candidate products
for i in range(243):
    for j in range(12,15):
        if attribute_levels[j] == df_product_combo.iloc[i,4]:
            df_dummy_all.iloc[j,i] = 1
        else:
            df_dummy_all.iloc[j,i] = 0

# Assign dummy variable of 1 for our brand for all candidate products
df_dummy_all.iloc[17] = 1

df_dummy_all

,Cand_1,Cand_2,Cand_3,Cand_4,Cand_5,Cand_6,Cand_7,Cand_8,Cand_9,Cand_10,...,Cand_234,Cand_235,Cand_236,Cand_237,Cand_238,Cand_239,Cand_240,Cand_241,Cand_242,Cand_243
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,1
9,1,1,1,0,0,0,0,0,0,1,...,0,1,1,1,0,0,0,0,0,0


Next, we can define a function that will help us calculate share for each possible candidate product.

In [453]:
def calc_share(cand_dum):
    # Initialize dummy variable matrix dataframe
    dum = np.zeros(shape=(18,3))
    df_dum = pd.DataFrame(dum, columns = ['Prod1','Prod2','Cand_Prod'])

    # Create dummy variable vectors for products 1 and 2
    dum_1 = [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
    dum_2 = [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]

    df_dum['Prod1'] = dum_1
    df_dum['Prod2'] = dum_2
    df_dum['Cand_Prod'] = cand_dum
    
    # Calculate utilities multiplied by our "c" value and then exponentiated
    util = np.dot(df_IPR,df_dum)
    c_util = np.multiply(util,c_value)
    exp_c_util = np.exp(c_util)

    # Use a for loop to find the sum of exponentiated utility values
    sum_ex = np.zeros(shape=(311,1)) 

    for i in range(len(sum_ex)):
        for j in range(3):
            sum_ex[i] += exp_c_util[i][j]

    # Use a for loop to get purchase probabilities and then compute average probabilities
    purc_prob = np.zeros(shape=(311,3))

    for i in range(len(sum_ex)):
        for j in range(3):
            purc_prob[i][j] = exp_c_util[i][j] / sum_ex[i]

    avg_probability = np.mean(purc_prob, axis=0)
    cand_share = avg_probability[2]
    return cand_share

Let's use the `calc_share` function we created to get share for each of our products now and add it into our table.

In [454]:
combo_share = []

for i in range(243):
    dummy_list = df_dummy_all.iloc[:,i].tolist()
    share = calc_share(dummy_list)
    combo_share.append(share)

df_combo['Share'] = combo_share
df_combo

,Price,Time,Capacity,Cleanability,Containment,Share,price,Margin,EPPP
1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant,0.016530,NaN,NaN,NaN
2,$30,0.5 hrs,12 oz,Difficult,Spill resistant,0.025234,NaN,NaN,NaN
3,$30,0.5 hrs,12 oz,Difficult,Leak resistant,0.042167,NaN,NaN,NaN
4,$30,0.5 hrs,12 oz,Fair,Slosh resistant,0.033560,NaN,NaN,NaN
5,$30,0.5 hrs,12 oz,Fair,Spill resistant,0.051275,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
239,$5,3 hrs,32 oz,Fair,Spill resistant,0.455497,NaN,NaN,NaN
240,$5,3 hrs,32 oz,Fair,Leak resistant,0.555399,NaN,NaN,NaN
241,$5,3 hrs,32 oz,Easy,Slosh resistant,0.441466,NaN,NaN,NaN
242,$5,3 hrs,32 oz,Easy,Spill resistant,0.529777,NaN,NaN,NaN


Next, let's create a function that will help us calculate the price, cost, and margin.

In [456]:
def calc_margin(cand_dum):
    # Compute price
    p_dum_arr = cand_dum.iloc[0:3].to_numpy()
    p_dum_arr = np.reshape(p_dum_arr, 3)

    cand_p = np.dot(p_array,p_dum_arr)

    # Compute cost
    c_dum_arr = cand_dum.iloc[3:15].to_numpy()
    c_dum_arr = np.reshape(c_dum_arr, 12)

    cand_c = np.dot(c_array,c_dum_arr)

    # Compute margin
    cand_m = cand_p - cand_c

    # Report our results below
    return cand_p, cand_m

Let's use the `calc_margin` function we created to get share for each of our products now and add it into our table. Then we can calculate EPPP by multiplying the margin value with the share value.

In [458]:
combo_price = []
combo_margin = []

for i in range(243):
    dummy = df_dummy_all.iloc[:,i]
    price, margin = calc_margin(dummy)
    combo_price.append(price)
    combo_margin.append(margin)

df_combo['price'] = combo_price
df_combo['Margin'] = combo_margin
df_combo['EPPP'] = df_combo['Share'] * df_combo['Margin']

Here, we have the final table with all 243 product combinations:

In [478]:
df_combo

,Price,Time,Capacity,Cleanability,Containment,Share,price,Margin,EPPP
1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant,0.016530,30,27.0,0.446321
2,$30,0.5 hrs,12 oz,Difficult,Spill resistant,0.025234,30,26.7,0.673761
3,$30,0.5 hrs,12 oz,Difficult,Leak resistant,0.042167,30,26.5,1.117416
4,$30,0.5 hrs,12 oz,Fair,Slosh resistant,0.033560,30,25.8,0.865858
5,$30,0.5 hrs,12 oz,Fair,Spill resistant,0.051275,30,25.5,1.307513
...,...,...,...,...,...,...,...,...,...
239,$5,3 hrs,32 oz,Fair,Spill resistant,0.455497,5,-3.8,-1.730890
240,$5,3 hrs,32 oz,Fair,Leak resistant,0.555399,5,-4.0,-2.221595
241,$5,3 hrs,32 oz,Easy,Slosh resistant,0.441466,5,-4.3,-1.898304
242,$5,3 hrs,32 oz,Easy,Spill resistant,0.529777,5,-4.6,-2.436975


## Question 3:

Let's identify the product with the highest expected profit per person by sorting our table.

In [484]:
sorted_eppp = df_combo.sort_values(by='EPPP', ascending=False)
sorted_eppp

,Price,Time,Capacity,Cleanability,Containment,Share,price,Margin,EPPP
72,$30,3 hrs,20 oz,Easy,Leak resistant,0.258840,30,20.4,5.280343
63,$30,3 hrs,12 oz,Easy,Leak resistant,0.226216,30,22.0,4.976757
81,$30,3 hrs,32 oz,Easy,Leak resistant,0.227583,30,20.2,4.597168
45,$30,1 hrs,20 oz,Easy,Leak resistant,0.194526,30,22.4,4.357385
36,$30,1 hrs,12 oz,Easy,Leak resistant,0.171599,30,24.0,4.118383
...,...,...,...,...,...,...,...,...,...
231,$5,3 hrs,20 oz,Fair,Leak resistant,0.615129,5,-3.8,-2.337492
242,$5,3 hrs,32 oz,Easy,Spill resistant,0.529777,5,-4.6,-2.436975
233,$5,3 hrs,20 oz,Easy,Spill resistant,0.585843,5,-4.4,-2.577708
243,$5,3 hrs,32 oz,Easy,Leak resistant,0.630500,5,-4.8,-3.026400


As we can see, our optimal product here is **product number 72** with the following attributes and metrics:<br>
- **Price**: $30
- **Time Insulation**: 3 hours
- **Capacity**: 20 oz
- **Cleanability**: Easy
- **Containment**: Leak resistant
- **Share**: 0.259
- **Margin**: 20.4
- **EPPP**: 5.28

## Question 4:

### Part 1)

The business rationale behind launching the product with the highest market share instead of the product with the highest EPPP would be if your company is looking to simply generate new customers for your business. By having larger market share, you could capture more people in your future market, and therefore increase your total sales volume as a result.

In [485]:
sorted_share = df_combo.sort_values(by='Share', ascending=False)
sorted_share

,Price,Time,Capacity,Cleanability,Containment,Share,price,Margin,EPPP
234,$5,3 hrs,20 oz,Easy,Leak resistant,0.696713,5,-4.6,-3.204878
225,$5,3 hrs,12 oz,Easy,Leak resistant,0.638187,5,-3.0,-1.914560
243,$5,3 hrs,32 oz,Easy,Leak resistant,0.630500,5,-4.8,-3.026400
207,$5,1 hrs,20 oz,Easy,Leak resistant,0.622748,5,-2.6,-1.619145
231,$5,3 hrs,20 oz,Fair,Leak resistant,0.615129,5,-3.8,-2.337492
...,...,...,...,...,...,...,...,...,...
37,$30,1 hrs,20 oz,Difficult,Slosh resistant,0.022077,30,24.9,0.549726
46,$30,1 hrs,32 oz,Difficult,Slosh resistant,0.019896,30,24.7,0.491441
1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant,0.016530,30,27.0,0.446321
10,$30,0.5 hrs,20 oz,Difficult,Slosh resistant,0.014843,30,25.4,0.377015


As we can see, our product with the highest share here is **product number 234** with the following attributes:<br>
- **Price**: 5
- **Time Insulation**: 3 hours
- **Capacity**: 20 oz
- **Cleanability**: Easy
- **Containment**: Leak resistant

### Part 2)

The business rationale behind launching the product with the highest margin would be if the firm is focused solely on maximizing their overall profit from each sale of their product. This is because profit of this product is based on both sales volume and profit margin, and having the highest profit margin would therefore be helpful to raising overall profits as well. 

In [488]:
sorted_margin = df_combo.sort_values(by='Margin', ascending=False)
sorted_margin

,Price,Time,Capacity,Cleanability,Containment,Share,price,Margin,EPPP,Cost
1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant,0.016530,30,27.0,0.446321,3.0
2,$30,0.5 hrs,12 oz,Difficult,Spill resistant,0.025234,30,26.7,0.673761,3.3
3,$30,0.5 hrs,12 oz,Difficult,Leak resistant,0.042167,30,26.5,1.117416,3.5
28,$30,1 hrs,12 oz,Difficult,Slosh resistant,0.023768,30,26.5,0.629841,3.5
29,$30,1 hrs,12 oz,Difficult,Spill resistant,0.035872,30,26.2,0.939847,3.8
...,...,...,...,...,...,...,...,...,...,...
241,$5,3 hrs,32 oz,Easy,Slosh resistant,0.441466,5,-4.3,-1.898304,9.3
233,$5,3 hrs,20 oz,Easy,Spill resistant,0.585843,5,-4.4,-2.577708,9.4
234,$5,3 hrs,20 oz,Easy,Leak resistant,0.696713,5,-4.6,-3.204878,9.6
242,$5,3 hrs,32 oz,Easy,Spill resistant,0.529777,5,-4.6,-2.436975,9.6


As we can see, our product with the highest margin here is **product number 1** with the following attributes:<br>
- **Price**: 30
- **Time Insulation**: 0.5 hours
- **Capacity**: 12 oz
- **Cleanability**: Difficult
- **Containment**: Slosh resistant

### Part 3)

The business rationale behind launching the product with the lowest cost would be if the company is looking to implement only a market-driven strategy rather than a market-driving strategy. By going with a market-driven strategy, the goal would be to just satisfy current customers through creating a new product for an already existing product category. In addition, having the lowest cost for the product helps to minimize risk for the firm if they are looking to generate some more money while limiting the possibility of failure for the new product.

In [487]:
df_combo["Cost"] = df_combo["price"] - df_combo["Margin"]
sorted_cost = df_combo.sort_values(by='Cost')
sorted_cost

,Price,Time,Capacity,Cleanability,Containment,Share,price,Margin,EPPP,Cost
1,$30,0.5 hrs,12 oz,Difficult,Slosh resistant,0.016530,30,27.0,0.446321,3.0
82,$10,0.5 hrs,12 oz,Difficult,Slosh resistant,0.105133,10,7.0,0.735932,3.0
163,$5,0.5 hrs,12 oz,Difficult,Slosh resistant,0.193544,5,2.0,0.387089,3.0
83,$10,0.5 hrs,12 oz,Difficult,Spill resistant,0.145872,10,6.7,0.977345,3.3
164,$5,0.5 hrs,12 oz,Difficult,Spill resistant,0.238293,5,1.7,0.405097,3.3
...,...,...,...,...,...,...,...,...,...,...
80,$30,3 hrs,32 oz,Easy,Spill resistant,0.147413,30,20.4,3.007224,9.6
72,$30,3 hrs,20 oz,Easy,Leak resistant,0.258840,30,20.4,5.280343,9.6
162,$10,3 hrs,32 oz,Easy,Leak resistant,0.542718,10,0.2,0.108544,9.8
81,$30,3 hrs,32 oz,Easy,Leak resistant,0.227583,30,20.2,4.597168,9.8


As we can see, our products with the lowest costs are **product numbers 1, 8, and 163** with the following attributes:<br>
- **Price**: 30, 10, or 5
- **Time Insulation**: 0.5 hours
- **Capacity**: 12 oz
- **Cleanability**: Difficult
- **Containment**: Slosh resistant